# Setup

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import helpers_05

%matplotlib inline   

D:\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


# Convolutions

In order to give our model more spatial awareness, provide translation invariance, and reduce the amount of parameters in our network, we need to share weights in such a way that the same pixels located in different parts of a image output identical activation values. The technique we end up using is called a _convolution_.

![](images/convolution_animated.gif)

The defining feature of a convolution is the _kernel_ (also known as a _filter_), which is a grid-like set of weights which slides over regions of an input image. At each step, the kernel weights are multiplied with the corresponding pixel values underneath. These multiplied values are then summed to get the output value at that point.

![](images/convolution_still.png)

By performing the operation in this way, we can use local spatial information while also making sure that each part of the image can be examined with the same weights.

This is something that is easier to learn visually, so let's take a look at a simple example:

![](images/basic_kernel_anim.gif)

Here, we have a 5x5 input matrix, and our kernel is 3x3. When the kernel is placed in the top-left portion of the input, we end up with the following total sum:

```
(1 * -1) + (2 * 1) + (0 * 2) + (1 * 1) + (0 * 1) + (0 * 0) + (2 * -1)+ (2 * 0) = -2
```

We can see that the value of `-2` is the top-left entry in our output matrix. As we slide the kernel across the image, we get corresponding outputs which follow the kernel spatially. 

## Manual Convolutions with NumPy

Let's do a quick example in NumPy/TensorFlow to verify that the above illustration works. First, we'll create our input matrix and kernels in NumPy:

In [2]:
# Example input matrix
a = np.array([[1, 2, 0, 3, 1],
              [1, 0, 0, 2, 2],
              [2, 1, 2, 1, 1],
              [0, 0, 1, 0, 0],
              [1, 2, 1, 1, 1]]).reshape(1,5,5,1).astype(np.float32)

# Example kernel
kernel = np.array([[-1,  1, 2],
                   [ 1,  1, 0],
                   [-1, -2, 0]]).reshape(3,3,1,1).astype(np.float32)

You can see that the values we set in our NumPy arrays match those in the visual above. A small implementation quirk is that we reshape the NumPy arrays.  The input matrix goes from `[5, 5]` to `[1, 5, 5, 1]`. The values, from left to right, represent the number of elements:
  * in the batch (1), 
  * the height (5), 
  * the width (5), and 
  * the number of channels (1). 
  
If our test matrix represented an RGB image, the shape would be `[1, 5, 5, 3]`. If there were 10 images in the batch, the shape would be `[10, 5, 5, 1]`

The kernel goes from `[3, 3]` to `[3, 3, 1, 1]`. The values, from left to right, represent:
  * the height of the kernel (3), 
  * the width of the kernel (3), 
  *  the number of channels in the _input_ (1), and 
  *  the number of channels in the output (1). 
  
If the input represented an RGB image, we'd have the shape `[3, 3, 3, 1]`, and if we wanted our output to have 5 channels, the output shape would be `[3, 3, 1, 5]`.  Think of the last two components as being similar to a weight matrix on a fully connected layer, which has the shape `[prev_num_neurons, curr_num_neurons]`.
    
Now that we have our dummy data, let's run it and see what we get!  

In [11]:
out = tf.Session().run(tf.nn.conv2d(input=a,
                                    filter=kernel,
                                    strides=[1, 1, 1, 1],
                                    padding='VALID'))
# np.squeeze() removes dimensions equal to `1` from a matrix/tensor
# The result of tf.conv2d is four dimensional, so this cleans it up
print(out.squeeze())

[[-2. -1.  3.]
 [ 2.  5.  8.]
 [-2.  0. -1.]]


The `tf.nn.conv2d()` Operation expects four main arguments: `input`, `filter`, `strides`, and `padding`:

* `input` is the input Tensor, in this case our `[1, 5, 5, 1]` test data.
* `filter` is the kernel. Typically, this would be a `tf.Variable`, but in this demo it's just our `[3,3,1,1]` kernel.

The other two inputs are things we haven't talked about yet, stride and padding.

##### Stride

"Stride" refers to the number of squares we move our kernel for each step (both vertically and horizontally). In the above example, both our strides (horizontal and vertical) were 1, but if they were set to 2, the operation would look like this:

![](images/basic_kernel_stride2_anim.gif)

One note:  we "walk" across using the horizontal stride (across columns) before we "reset" back to the left with the vertical stride (down rows).  This is almost like an old-school typewriter with a *ding* when you get to the end of the line.

Because we're skipping over a square in both directions, our output `Tensor` is 2x2 instead of 3x3. This is one way to reduce the spatial dimensions of a network- we'll look at another technique for this, pooling, later in this lesson.

The way we set the stride in TensorFlow is with the `stride` parameter, which is a list of integers with the form `[1, vert_stride, hori_stride, 1]` (the first and last elements have to be `1`. We can test this out by modifying the code from above:

In [14]:
out = tf.Session().run(tf.nn.conv2d(input=a,
                                    filter=kernel,
                                    strides=[1, 2, 2, 1],
                                    padding='VALID'))
print(out.squeeze())

[[-2.  3.]
 [-2. -1.]]


##### Padding

You may have noticed that we ended up reducing the spatial dimensions from our input to the output: we start with a `5x5` Tensor, and end up with a `3x3` Tensor as output. While this is sometimes ok, it's often the case that we want to maintain the width and height of out input. We can achieve this by adding zeros around our image or *padding* the image. By adding a ring of one or more zeros to the outside of our input and letting our kernel "overflow" on the sides, we can ensure that the output dimensions match the input dimensions.

![](images/padded_kernel_anim.gif)

_Side note: the amount of zeros needed on the outside of the input depends on the size of the kernel: `3x3` needs one layer of zeros, `5x5` needs two, `7x7` needs three, etc._

We can control padding in TensorFlow by using the `padding` parameter. `padding` takes a string which selects between two options:

* `'VALID'` is what we used above - no zero padding. The idea is that we're only using "real" or "valid" input data to get outputs. You could also think of it as the kernel not going "out of bounds", thus staying "valid"
* `'SAME'` uses zero-padding to keep the output dimensions equal to that of the input (assuming horizontal and vertical stride is set to 1). This one is more self-explanatory: we're keeping the dimensions the "same"

_Side note: this naming scheme comes from the convolution syntax in [NumPy](https://docs.scipy.org/doc/numpy/reference/generated/numpy.convolve.html) and [MATLAB](https://www.mathworks.com/help/matlab/ref/conv2.html)_

Let's try out `'SAME'` padding with our dummy data:

In [15]:
out = tf.Session().run(tf.nn.conv2d(input=a,
                                    filter=kernel,
                                    strides=[1, 1, 1, 1],
                                    padding='SAME'))
print(out.squeeze())

[[-1.  2.  2. -1. -2.]
 [ 2. -2. -1.  3. -1.]
 [ 3.  2.  5.  8.  2.]
 [ 2. -2.  0. -1. -3.]
 [ 1.  5.  4.  1.  2.]]


## Multiple layers of input and kernels

![](images/two_layer_kernel_anim.gif)

![](images/two_layer_kernel_final.png)

##### Adding a Bias

We will also add in a bias term for convolutions the same way we would a fully-connected layer. Each layer of the filter will have a constant number that is added along with the rest of the kernel dot product. We create a `Variable` vector with the same depth as the output `Tensor` (the last number in the shape of the kernel). Then, we use [`tf.nn.bias_add()`](https://www.tensorflow.org/versions/master/api_docs/python/nn/activation_functions_#bias_add) to add the value the output of our convolution.

```
conv = tf.nn.conv2d(input, kernel, ...)
bias = tf.Variable(tf.constant(0.1, shape=[depth]))
total = tf.nn.bias(conv, bias)
```

For convolutions, the bias is usually initialized to some small positive value (as opposed to zero). We do this because the general go-to activation function for CNNs is ReLU, which can "die" if they never get positive inputs.

##### Small Aside About Kernels

[Kernels](https://en.wikipedia.org/wiki/Kernel_(image_processing) have been used in image processing for many years for tasks such as edge detection, blurring, and sharpening. The basic idea of convolutional neural networks is that we can let the computer find useful kernels for the task instead of using handmade, pre-defined ones.

# Pooling

Earlier, we saw that using a higher stride convolution results in collapsing the spatial dimensions of our network. While it is sometimes advantageous to do this during trained convolutions, a more common technique is to use _pooling_, which does a transformation on neighboring inputs (but is not trained/has zero parameters).

## Max Pooling

In max pooling, the network looks at a chunk of neighboring pixels and outputs the largest pixel from that group:

![Max pooling, size 2, stride 2](images/maxpool.png)

In this example, we've split a `4x4` matrix into four groups (shown by  colors), each holding four numbers. We take the largest number from each group to output the matrix on the right. To demonstrate this in TensorFlow, we'll use the [`tf.nn.max_pool()`](https://www.tensorflow.org/versions/master/api_docs/python/nn/pooling#max_pool) Op. Because each group is a `[2x2]` box, and there are 2 steps between boxes (horizontally and vertically), we say that this is a max pool, size 2, stride 2.

In [16]:
# Setup our input array
pool_me = np.array([[ 2,  1,  0, -1],
                    [-3,  8,  2,  5],
                    [ 1, -1,  3,  4],
                    [ 0,  1,  1, -2]]).reshape(1,4,4,1).astype(np.float32)

out = tf.Session().run(tf.nn.max_pool(value=pool_me,
                                      ksize=[1, 2, 2, 1],
                                      strides=[1, 2, 2, 1],
                                      padding='VALID'))
print(out.squeeze())

[[8. 5.]
 [1. 4.]]


You'll notice that max pooling has similar parameters to `tf.nn.conv2d()`. The primary difference is `ksize` instead of `filter`: `ksize` is a list of integers representing the shape of the pooling kernel with shape `[1, k_height, k_width, 1]`. In this case, it's `[1, 2, 2, 1]`, as our kernel is two squares high and two squares wide.

`stride` and `padding` act the same as they do for convolutions. In general the `strides` option is the same or larger as `ksize` (no overlapping kernels), and `padding` is generally set to `'VALID'`.

## Average Pooling

Average pooling is the same idea as max pooling, only instead we take the average of all the values in each group.

![Average pooling, size 2, stride 2](images/avgpool.png)

We can use average pooling with the [`tf.nn.avg_pool`](https://www.tensorflow.org/versions/master/api_docs/python/nn/pooling#avg_pool) Op:

In [17]:
out = tf.Session().run(tf.nn.avg_pool(value=pool_me,
                                      ksize=[1, 2, 2, 1],
                                      strides=[1, 2, 2, 1],
                                      padding='VALID'))
print(out.squeeze())

[[2.   1.5 ]
 [0.25 1.5 ]]


## Max or Avg Pool?

The jury is still out as to whether or not there is a definitive answer whether max pooling or average pooling is more effective. At this time, I'd say max pooling is more common, though average pooling is used in several well-regarded modern models.

You can read a thorough discussion in [_A Theoretical Analysis of Feature Pooling in Visual Recognition_ (Boureau et al)](http://people.ee.duke.edu/~lcarin/icml2010b.pdf) if you want to dive deeper into the debate!

# LeNet

[FIXME: Add text.]  Our goal now is to implement LeNet.  

In [2]:
# we're going to make use a few names that we defined last week
# they've been gathered in helpers_05.py
from helpers_05 import (batches, flatten, fully_connected_layer,
                        fully_connected_sigmoid_layer,
                        test_and_show_images)

## Convolution Layers

[FIXME:  add text]

In [3]:
def conv_layer(incoming, num_kernels, kernel_sz, 
               strides=[1, 1, 1, 1], padding='SAME',
               bval=0.01, 
               activation_fn=tf.nn.relu, 
               name=None):
    prev_outshape = incoming.shape.dims[-1].value
    kshape = kernel_sz + [prev_outshape, num_kernels]

    fan_in = np.prod(incoming.shape[1:]).value
    xavier_stddev = np.sqrt(2.0 / fan_in)
    
    with tf.variable_scope(name, 'conv_layer'):
        w = tf.Variable(tf.truncated_normal(kshape, stddev=xavier_stddev), name='kernel')
        b = tf.Variable(tf.constant(bval, shape=[num_kernels]), name='bias')
        conv = tf.nn.conv2d(incoming, w, strides, padding, name='conv')
        z = tf.nn.bias_add(conv, b)
        return z if activation_fn is None else activation_fn(z)

In [4]:
def pool_layer(incoming, ksize, strides=None, padding='VALID',
               pool_fn=tf.nn.max_pool, name=None):
    'create a pooling layer:  we auto-add the leading/trailing 1s'
    ksize = [1] + ksize + [1]
    # default strides to ksize
    strides = strides if strides is not None else ksize
    with tf.variable_scope(name, 'pool_layer'):
        return pool_fn(incoming, ksize, strides, padding)

## LeNet Sub-Networks

To get us warmed up, here is a simple network that shares some of the architecture of LeNet, but it is smaller and it uses sigmoid activations.

In [5]:
def not_lenet(images):
    with tf.name_scope('small_model'):
        conv_1 = conv_layer(images, 6, [5, 5])
        print(tf.Session().run(conv_1).shape)
        pool_1 = pool_layer(conv_1, [2, 2])
        print(tf.Session().run(pool_1).shape)
        conv_2 = conv_layer(pool_1, 16, [5, 5])
        print(tf.Session().run(conv_2).shape)
        pool_2 = pool_layer(conv_2, [2, 2])
        print(tf.Session().run(pool_2).shape)
        
        flat = flatten(pool_2)
        
        fc_layer = fully_connected_sigmoid_layer
        fc_1 = fc_layer(flat, 120)
        fc_2 = fc_layer(fc_1,  84)
        fc_3 = fully_connected_layer(fc_2,  10, w_stddev = 0.5, activation_fn=None)
    
    return fc_3 

In [53]:
my_img = np.array([[ [2,0,0],  [1,0,0],  [0,0,0], [-1,0,0]],
                    [[-3,0,0],  [8,0,0],  [2,0,0],  [5,0,0]],
                    [ [1,0,0], [-1,0,0],  [3,0,0],  [4,0,0]],
                    [ [0,0,0],  [1,0,0],  [1,0,0], [-2,0,0]]]).reshape(1,4,4,3).astype(np.float32)
conv_1 = conv_layer(tf.constant(my_img), 6, [2, 2])
pool_1 = pool_layer(conv_1, [2, 2])
conv_2 = conv_layer(pool_1, 16, [2, 2])
pool_2 = pool_layer(conv_2,[2,2])
print(conv_1)
print(pool_1)
print(conv_2)
print(pool_2)

Tensor("conv_layer_2/Relu:0", shape=(1, 4, 4, 6), dtype=float32)
Tensor("pool_layer_2/MaxPool:0", shape=(1, 2, 2, 6), dtype=float32)
Tensor("conv_layer_3/Relu:0", shape=(1, 2, 2, 16), dtype=float32)
Tensor("pool_layer_3/MaxPool:0", shape=(1, 1, 1, 16), dtype=float32)


We very briefly discussed Xavier initialization in the slides.  It isn't that hard to implement, either.  Here, we simply add a value to compute `w_stddev` instead of setting it to a fixed constant (see line 8 in the next cell).

In [7]:
def fully_connected_xavier_relu_layer(incoming_layer, num_nodes,
                                      b_val=0.01,
                                      keep_prob=None, name=None):
    ' pass through for fully_connected_layer with xavier init '
    incoming_layer = tf.convert_to_tensor(incoming_layer)
    prev_num_nodes = incoming_layer.shape.dims[-1].value
    
    w_stddev = np.sqrt(2.0 / prev_num_nodes)

    return fully_connected_layer(incoming_layer, num_nodes,
                                 w_stddev = w_stddev, b_val=b_val,
                                 activation_fn = tf.nn.relu,
                                 keep_prob=keep_prob,
                                 name=name)


With the above as a starting point, we can now recrete that actual LeNet.  Here's the table from the slides:

![](images/lenet-table.png)

and the graphic from the paper:

![](images/lenet.png)

Unfortunately, you have to combine information from the table and the graphic to get the numbers below.  The depths come from the graphic (they precede the `@` signs on the top of the figure).  The other values come from the table.

In [8]:
def lenet_small(images):
    with tf.name_scope('small_lenet_model'):    
        conv_1 = conv_layer(images, 6, [5, 5])
        print()
        pool_1 = pool_layer(conv_1, [2, 2])
        conv_2 = conv_layer(pool_1, 16, [5, 5])
        pool_2 = pool_layer(conv_2, [2, 2])

        flat = flatten(pool_2)

        fc_layer = fully_connected_xavier_relu_layer
        fc_1   = fc_layer(flat, 120)
        fc_2   = fc_layer(fc_1, 84)
        fc_3   = fc_layer(fc_2, 10)
        fc_4   = fully_connected_layer(fc_3, 10, activation_fn=None)
    
    return fc_4

And here's a scaled up LeNet:

In [9]:
def lenet_big(images):
    with tf.name_scope('big_lenet_model'):    
        conv_1 = conv_layer(images, 32, [5, 5])
        pool_1 = pool_layer(conv_1, [2, 2])
        conv_2 = conv_layer(pool_1, 64, [5, 5])
        pool_2 = pool_layer(conv_2, [2, 2])

        flat = flatten(pool_2)

        fc_layer = fully_connected_xavier_relu_layer
        fc_1   = fc_layer(flat, 400)
        fc_2   = fc_layer(fc_1, 200)
        fc_3   = fc_layer(fc_2, 200)
        fc_4   = fully_connected_layer(fc_3, 10, activation_fn=None)
    
    return fc_4

# Exercise

You mission, should you choose to accept it, is to take the pieces we've developed and turn them into a working model for Lenet on MNIST.  Here's a process to get you started:

  1. Grab either your CIFAR or MNIST model from last week.
  2.  Modify it so that your s.out is coming from one of the LeNet versions we defined above.
  3.  Modify your `training` block so it uses a slightly different optimizer (we will talk about these in the coming weeks).
```
    with tf.name_scope('train'):
        decayed_rate = tf.train.exponential_decay(s.learning_rate, global_step,
                                              600, 0.998, True)
        momopt = tf.train.MomentumOptimizer
        s.train = momopt(decayed_rate, 0.9).minimize(s.loss)
```
  4.  Read in your training and testing data with `helpers_05.get_mnist_dataset` (like we did last week)
  5.  Setup a training loop like we did last week.  Note, use a `learning_rate=0.001` and we don't need a `momentum` (which we used last time).
  6.  If things are working right, you shouldn't need 15 epochs of training.  Five or six should do.
  7.  Good luck!  Feel free to consult the solution below.

## Solution

In [24]:
(train_data,train_labels,test_data,test_labels)=helpers_05.get_mnist_dataset()

In [42]:
train_data.shape
test_labels.shape

(10000,)

In [26]:
class MNIST_Model():
    def __init__(s):
        graph = tf.Graph()
        with graph.as_default():
            with tf.name_scope("input"):
                s.x = tf.placeholder(tf.uint8,shape=[None,28,28,1],name='x')
                s.y = tf.placeholder(tf.int32,shape=[None],name='y')
            with tf.name_scope("hyperparams"):
                s.learning_rate = tf.placeholder(tf.float32,[],name='learning_rate')
                s.x_keep_prob = tf.placeholder(tf.float32,[],name='x_keep_prob')
                s.h_keep_prob = tf.placeholder(tf.float32,[],name='h_keep_prob')
            with tf.name_scope("preprocess"):
                s.x_float = tf.cast(s.x,tf.float32)
                s.x_dropped = tf.nn.dropout(s.x_float,keep_prob=s.x_keep_prob)
                s.one_hot_lables = tf.one_hot(s.y,10)
            with tf.name_scope("model"):
                s.conv1 = conv_layer(s.x_dropped,6,[5,5])
                s.pool1 = pool_layer(s.conv1,[2,2])
                s.conv2 = conv_layer(s.pool1,16,[5,5])
                s.pool2 = pool_layer(s.conv2,[2,2])
                flat = flatten(s.pool2)
                fc_layer = fully_connected_xavier_relu_layer
                s.fc1 = fc_layer(flat, 120,keep_prob=s.h_keep_prob)
                s.fc2 = fc_layer(s.fc1, 84,keep_prob=s.h_keep_prob)
                s.out = fc_layer(s.fc2, 10,keep_prob=s.h_keep_prob)
                #s.out = fully_connected_layer(s.fc3, 10, activation_fn=None)
            with tf.name_scope("loss"):
                smce = tf.nn.softmax_cross_entropy_with_logits_v2
                s.loss = tf.reduce_mean(smce(logits = s.out,labels = s.one_hot_lables))
            with tf.name_scope("global_step"):
                global_step = tf.Variable(0,trainable=False,name="global_step" )
                s.inc_step = tf.assign(global_step,global_step+1,name="inc_step")
            with tf.name_scope("train"):
                decayed_rate = tf.train.exponential_decay(s.learning_rate, global_step,
                                        600, 0.998, True)
                momopt = tf.train.MomentumOptimizer(decayed_rate,0.9)
                s.train = momopt.minimize(s.loss)
            with tf.name_scope("prediction"):
                s.softmax = tf.nn.softmax(s.out,name="softmax")
                s.prediction = tf.cast(tf.argmax(s.softmax,1),tf.int32)
                s.correct = tf.equal(s.prediction,s.y)
                s.accuracy = tf.reduce_mean(tf.cast(s.correct,tf.float32)) 
            init = tf.global_variables_initializer()
        s.sess = tf.Session(graph=graph)
        s.sess.run(init)
    
    def fit(s,train_dict):
        tr_loss,step,tr_acc,_ = s.sess.run([s.loss,s.inc_step,s.accuracy,s.train],feed_dict=train_dict)
        return tr_loss,step,tr_acc,_
    
    def predict(s,test_dict):
        ct_correct,pred = s.sess.run([s.correct,s.prediction],feed_dict=test_dict)
        return ct_correct,pred
        
                

In [34]:
CNN_mm = MNIST_Model()

for epoch in range(10):
    for batch_data,batch_labels in batches(train_data,train_labels,100):
        train_dict ={CNN_mm.x:batch_data,
                     CNN_mm.y:batch_labels,
                     CNN_mm.learning_rate:0.001,
                     CNN_mm.x_keep_prob:0.8,
                     CNN_mm.h_keep_prob:1
        }
        tr_loss,step,tr_acc,_ = CNN_mm.fit(train_dict)
    print(f"epoch: {epoch} step: {step} loss: {tr_loss} acc: {tr_acc}")

epoch: 0 step: 600 loss: 0.7272697687149048 acc: 0.7400000095367432
epoch: 1 step: 1200 loss: 0.6253323554992676 acc: 0.75
epoch: 2 step: 1800 loss: 0.8116852045059204 acc: 0.6899999976158142
epoch: 3 step: 2400 loss: 0.5320529341697693 acc: 0.800000011920929
epoch: 4 step: 3000 loss: 0.16889211535453796 acc: 0.9399999976158142
epoch: 5 step: 3600 loss: 0.05312144383788109 acc: 0.9900000095367432
epoch: 6 step: 4200 loss: 0.13868233561515808 acc: 0.9599999785423279
epoch: 7 step: 4800 loss: 0.0440414696931839 acc: 0.9800000190734863
epoch: 8 step: 5400 loss: 0.05750611424446106 acc: 0.9900000095367432
epoch: 9 step: 6000 loss: 0.042615074664354324 acc: 0.9700000286102295


In [35]:
#get test acc
batch_correct_cts = []
for batch_data, batch_labels in batches(test_data, test_labels, 200):
    test_dict = {CNN_mm.x : batch_data,   CNN_mm.x_keep_prob : 1.0,
                 CNN_mm.y : batch_labels, CNN_mm.h_keep_prob : 1.0}
    correctness, curr_preds = CNN_mm.predict(test_dict)

    batch_correct_cts.append(correctness.sum())

print(sum(batch_correct_cts) / len(test_data))

0.9779


# Optional Task

Now that we've got a more powerful network architecture, let's try to do better on CIFAR.  Take your new and improved LeNet model and point it at the CIFAR dataset.

In [38]:
import helpers_04
(train_data, train_labels,
            test_data, test_labels), readable_labels = helpers_04.load_cifar()
#when you run the code you may encounter errors,just copy the floder "data/cifar-10-batches-py" to folder "data/cifar"

attempting to download cifar
Extracting data for data/cifar\cifar-10-python. This may take a while. Please wait.


In [41]:
train_data,test_data=train_data.reshape((-1,32,32,3)),test_data.reshape((-1,32,32,3))

In [48]:
np.unique(train_labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [59]:
class CIFAR_Model():
    def __init__(s):
        graph = tf.Graph()
        with graph.as_default():
            with tf.name_scope("input"):
                s.x = tf.placeholder(tf.uint8,shape=[None,32,32,3],name='x')
                s.y = tf.placeholder(tf.int32,shape=[None],name='y')
            with tf.name_scope("hyperparams"):
                s.learning_rate = tf.placeholder(tf.float32,[],name='learning_rate')
                s.x_keep_prob = tf.placeholder(tf.float32,[],name='x_keep_prob')
                s.h_keep_prob = tf.placeholder(tf.float32,[],name='h_keep_prob')
            with tf.name_scope("preprocess"):
                s.x_float = tf.cast(s.x,tf.float32)
                s.x_dropped = tf.nn.dropout(s.x_float,keep_prob=s.x_keep_prob)
                s.one_hot_lables = tf.one_hot(s.y,10)
            with tf.name_scope("model"):
                s.conv1 = conv_layer(s.x_dropped,32,[5,5])
                s.pool1 = pool_layer(s.conv1,[2,2])
                s.conv2 = conv_layer(s.pool1,64,[5,5])
                s.pool2 = pool_layer(s.conv2,[2,2])
                flat = flatten(s.pool2)
                fc_layer = fully_connected_xavier_relu_layer
                s.fc1 = fc_layer(flat, 600,keep_prob=s.h_keep_prob)
                s.fc2 = fc_layer(s.fc1, 300,keep_prob=s.h_keep_prob)
                s.fc3 = fc_layer(s.fc2, 300,keep_prob=s.h_keep_prob)
                s.out = fully_connected_layer(s.fc3, 10, activation_fn=None)
            with tf.name_scope("loss"):
                smce = tf.nn.softmax_cross_entropy_with_logits_v2
                s.loss = tf.reduce_mean(smce(logits = s.out,labels = s.one_hot_lables))
            with tf.name_scope("global_step"):
                global_step = tf.Variable(0,trainable=False,name="global_step" )
                s.inc_step = tf.assign(global_step,global_step+1,name="inc_step")
            with tf.name_scope("train"):
                decayed_rate = tf.train.exponential_decay(s.learning_rate, global_step,
                                        600, 0.998, True)
                momopt = tf.train.MomentumOptimizer(decayed_rate,0.9)
                s.train = momopt.minimize(s.loss)
            with tf.name_scope("prediction"):
                s.softmax = tf.nn.softmax(s.out,name="softmax")
                s.prediction = tf.cast(tf.argmax(s.softmax,1),tf.int32)
                s.correct = tf.equal(s.prediction,s.y)
                s.accuracy = tf.reduce_mean(tf.cast(s.correct,tf.float32)) 
            init = tf.global_variables_initializer()
        s.sess = tf.Session(graph=graph)
        s.sess.run(init)
    
    def fit(s,train_dict):
        tr_loss,step,tr_acc,_ = s.sess.run([s.loss,s.inc_step,s.accuracy,s.train],feed_dict=train_dict)
        return tr_loss,step,tr_acc,_
    
    def predict(s,test_dict):
        ct_correct,pred = s.sess.run([s.correct,s.prediction],feed_dict=test_dict)
        return ct_correct,pred
        
                

In [62]:
CNN_cm = CIFAR_Model()

for epoch in range(60):
    for batch_data,batch_labels in batches(train_data,train_labels,100):
        train_dict ={CNN_cm.x:batch_data,
                     CNN_cm.y:batch_labels,
                     CNN_cm.learning_rate:0.001,
                     CNN_cm.x_keep_prob:1,
                     CNN_cm.h_keep_prob:1
        }
        tr_loss,step,tr_acc,_ = CNN_cm.fit(train_dict)
    print(f"epoch: {epoch} step: {step} loss: {tr_loss} acc: {tr_acc}")

epoch: 0 step: 500 loss: 2.3004088401794434 acc: 0.10999999940395355
epoch: 1 step: 1000 loss: 2.3015239238739014 acc: 0.1599999964237213
epoch: 2 step: 1500 loss: 2.305943727493286 acc: 0.07000000029802322
epoch: 3 step: 2000 loss: 2.293351650238037 acc: 0.14000000059604645
epoch: 4 step: 2500 loss: 2.1881067752838135 acc: 0.23999999463558197
epoch: 5 step: 3000 loss: 1.6883143186569214 acc: 0.44999998807907104
epoch: 6 step: 3500 loss: 1.5064512491226196 acc: 0.44999998807907104
epoch: 7 step: 4000 loss: 1.7089828252792358 acc: 0.4699999988079071
epoch: 8 step: 4500 loss: 1.6168855428695679 acc: 0.4099999964237213
epoch: 9 step: 5000 loss: 1.3416905403137207 acc: 0.5400000214576721
epoch: 10 step: 5500 loss: 1.301720380783081 acc: 0.550000011920929
epoch: 11 step: 6000 loss: 1.2371163368225098 acc: 0.5
epoch: 12 step: 6500 loss: 1.0548182725906372 acc: 0.6499999761581421
epoch: 13 step: 7000 loss: 1.1199522018432617 acc: 0.6100000143051147
epoch: 14 step: 7500 loss: 1.124638795852661

In [63]:
#get test acc
batch_correct_cts = []
for batch_data, batch_labels in batches(test_data, test_labels, 200):
    test_dict = {CNN_cm.x : batch_data,   CNN_cm.x_keep_prob : 1.0,
                 CNN_cm.y : batch_labels, CNN_cm.h_keep_prob : 1.0}
    correctness, curr_preds = CNN_cm.predict(test_dict)

    batch_correct_cts.append(correctness.sum())

print(sum(batch_correct_cts) / len(test_data))

0.4764
